In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim

from src.models import CifarResNet, MNIST_CNN, CIFAR_CNN
from src.helpers import evaluate_rob_accuracy, evaluate_clean_accuracy, load_model, safe_model,_evaluate_model
from src.data_loader import load_torchvision_dataset, load_imagenette
#from src.pruning import identify_layers, _evaluate_sparsity

import time

if torch.cuda.is_available() == True:
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
print(device)
dtype = torch.float32

cuda:0


In [2]:
def load_torchvision_dataset(dataset, batchsize=512, data_augmentation=False):
    if data_augmentation == True:
        train_transforms = torchvision.transforms.Compose([
            torchvision.transforms.ColorJitter(hue=.05, saturation=.05),
            torchvision.transforms.RandomHorizontalFlip(),
            torchvision.transforms.RandomRotation(20),
            torchvision.transforms.Resize(40),
            torchvision.transforms.RandomResizedCrop(32),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),        
        ])
    if data_augmentation == False:
        train_transforms = torchvision.transforms.Compose([
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),        
        ])
    val_transforms = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
    
    if dataset == 'MNIST':
        train = torchvision.datasets.MNIST('./data', train=True, transform=torchvision.transforms.ToTensor(), download=True)
        test = torchvision.datasets.MNIST('./data', train=False, transform=torchvision.transforms.ToTensor(), download=True)
    if dataset == 'CIFAR10':
        train = torchvision.datasets.CIFAR10('./data', train=True, transform=train_transforms, download=True)
        test = torchvision.datasets.CIFAR10('./data', train=False, transform=val_transforms, download=True)
    train_loader = torch.utils.data.DataLoader(
        train,
        batch_size=batchsize,
        shuffle=True,
    )
    test_loader = torch.utils.data.DataLoader(
        test,
        batch_size=batchsize,
        shuffle=True,
    )
    return train_loader, test_loader

train_loader, test_loader = load_torchvision_dataset('CIFAR10', batchsize=128, data_augmentation=False)

Files already downloaded and verified
Files already downloaded and verified


# No Augmentation - 108 s

In [8]:
#model = CifarResNet().to(device)
model.fit(train_loader, test_loader, 3, device)

[1,     1] loss: 0.87525, train_accuracy: 71.09
[1,     2] loss: 1.81754, train_accuracy: 53.12
[1,     3] loss: 2.90354, train_accuracy: 48.44
[1,     4] loss: 2.27259, train_accuracy: 40.62
[1,     5] loss: 1.57684, train_accuracy: 51.56
[1,     6] loss: 1.88914, train_accuracy: 49.22
[1,     7] loss: 2.06021, train_accuracy: 40.62
[1,     8] loss: 1.46237, train_accuracy: 57.81
[1,     9] loss: 1.31716, train_accuracy: 60.94
[1,    10] loss: 2.11844, train_accuracy: 54.69
[1,    11] loss: 1.64806, train_accuracy: 53.12
[1,    12] loss: 1.81902, train_accuracy: 57.03
[1,    13] loss: 1.64876, train_accuracy: 57.81
[1,    14] loss: 1.52725, train_accuracy: 55.47
[1,    15] loss: 1.52262, train_accuracy: 53.91
[1,    16] loss: 1.74966, train_accuracy: 43.75
[1,    17] loss: 1.17733, train_accuracy: 57.03
[1,    18] loss: 1.78217, train_accuracy: 46.09
[1,    19] loss: 2.01632, train_accuracy: 48.44
[1,    20] loss: 1.36680, train_accuracy: 58.59
[1,    21] loss: 1.29572, train_accuracy

[1,   172] loss: 1.02279, train_accuracy: 64.84
[1,   173] loss: 0.97144, train_accuracy: 71.09
[1,   174] loss: 0.63283, train_accuracy: 80.47
[1,   175] loss: 0.59900, train_accuracy: 77.34
[1,   176] loss: 0.96610, train_accuracy: 67.97
[1,   177] loss: 0.87859, train_accuracy: 72.66
[1,   178] loss: 0.95553, train_accuracy: 65.62
[1,   179] loss: 0.69874, train_accuracy: 76.56
[1,   180] loss: 0.80651, train_accuracy: 72.66
[1,   181] loss: 0.96836, train_accuracy: 63.28
[1,   182] loss: 0.75550, train_accuracy: 72.66
[1,   183] loss: 1.05013, train_accuracy: 67.97
[1,   184] loss: 0.81823, train_accuracy: 70.31
[1,   185] loss: 0.88716, train_accuracy: 70.31
[1,   186] loss: 0.92622, train_accuracy: 70.31
[1,   187] loss: 0.97265, train_accuracy: 66.41
[1,   188] loss: 0.54613, train_accuracy: 79.69
[1,   189] loss: 0.84934, train_accuracy: 73.44
[1,   190] loss: 0.80076, train_accuracy: 71.88
[1,   191] loss: 0.94644, train_accuracy: 67.97
[1,   192] loss: 0.94544, train_accuracy

[1,   343] loss: 0.58122, train_accuracy: 79.69
[1,   344] loss: 0.59925, train_accuracy: 82.03
[1,   345] loss: 0.74389, train_accuracy: 72.66
[1,   346] loss: 0.73941, train_accuracy: 73.44
[1,   347] loss: 0.64701, train_accuracy: 79.69
[1,   348] loss: 0.64721, train_accuracy: 77.34
[1,   349] loss: 0.67092, train_accuracy: 77.34
[1,   350] loss: 0.68900, train_accuracy: 75.00
[1,   351] loss: 0.70238, train_accuracy: 78.91
[1,   352] loss: 0.83198, train_accuracy: 75.00
[1,   353] loss: 0.77339, train_accuracy: 75.00
[1,   354] loss: 0.69830, train_accuracy: 75.78
[1,   355] loss: 0.68953, train_accuracy: 75.00
[1,   356] loss: 0.94624, train_accuracy: 67.97
[1,   357] loss: 0.95821, train_accuracy: 65.62
[1,   358] loss: 0.85500, train_accuracy: 72.66
[1,   359] loss: 0.63542, train_accuracy: 83.59
[1,   360] loss: 0.69774, train_accuracy: 72.66
[1,   361] loss: 0.80361, train_accuracy: 73.44
[1,   362] loss: 0.80438, train_accuracy: 72.66
[1,   363] loss: 0.66535, train_accuracy

[2,   121] loss: 0.73784, train_accuracy: 75.78
[2,   122] loss: 0.74558, train_accuracy: 75.78
[2,   123] loss: 0.51246, train_accuracy: 85.16
[2,   124] loss: 0.61427, train_accuracy: 71.88
[2,   125] loss: 0.45424, train_accuracy: 85.16
[2,   126] loss: 0.54579, train_accuracy: 80.47
[2,   127] loss: 0.68289, train_accuracy: 77.34
[2,   128] loss: 0.43115, train_accuracy: 85.94
[2,   129] loss: 0.43546, train_accuracy: 82.81
[2,   130] loss: 0.68373, train_accuracy: 75.00
[2,   131] loss: 0.66353, train_accuracy: 80.47
[2,   132] loss: 0.64222, train_accuracy: 78.12
[2,   133] loss: 0.67675, train_accuracy: 82.81
[2,   134] loss: 0.54615, train_accuracy: 81.25
[2,   135] loss: 0.80153, train_accuracy: 79.69
[2,   136] loss: 0.87810, train_accuracy: 67.19
[2,   137] loss: 0.52501, train_accuracy: 81.25
[2,   138] loss: 0.48927, train_accuracy: 80.47
[2,   139] loss: 0.70933, train_accuracy: 68.75
[2,   140] loss: 0.76757, train_accuracy: 75.00
[2,   141] loss: 0.66439, train_accuracy

[2,   292] loss: 0.52257, train_accuracy: 82.81
[2,   293] loss: 0.72770, train_accuracy: 76.56
[2,   294] loss: 0.49523, train_accuracy: 81.25
[2,   295] loss: 0.54752, train_accuracy: 80.47
[2,   296] loss: 0.55750, train_accuracy: 79.69
[2,   297] loss: 0.62624, train_accuracy: 75.78
[2,   298] loss: 0.39854, train_accuracy: 88.28
[2,   299] loss: 0.48476, train_accuracy: 83.59
[2,   300] loss: 0.70649, train_accuracy: 77.34
[2,   301] loss: 0.57118, train_accuracy: 80.47
[2,   302] loss: 0.51269, train_accuracy: 81.25
[2,   303] loss: 0.54638, train_accuracy: 81.25
[2,   304] loss: 0.65902, train_accuracy: 75.78
[2,   305] loss: 0.57479, train_accuracy: 80.47
[2,   306] loss: 0.62878, train_accuracy: 78.12
[2,   307] loss: 0.81359, train_accuracy: 72.66
[2,   308] loss: 0.55606, train_accuracy: 81.25
[2,   309] loss: 0.70758, train_accuracy: 78.91
[2,   310] loss: 0.63437, train_accuracy: 82.03
[2,   311] loss: 0.76427, train_accuracy: 74.22
[2,   312] loss: 0.64867, train_accuracy

[3,    71] loss: 0.56791, train_accuracy: 77.34
[3,    72] loss: 0.54192, train_accuracy: 82.03
[3,    73] loss: 0.49290, train_accuracy: 82.81
[3,    74] loss: 0.45257, train_accuracy: 86.72
[3,    75] loss: 0.27665, train_accuracy: 89.06
[3,    76] loss: 0.50973, train_accuracy: 82.03
[3,    77] loss: 0.57263, train_accuracy: 81.25
[3,    78] loss: 0.45637, train_accuracy: 85.94
[3,    79] loss: 0.44540, train_accuracy: 81.25
[3,    80] loss: 0.46824, train_accuracy: 83.59
[3,    81] loss: 0.45887, train_accuracy: 80.47
[3,    82] loss: 0.45769, train_accuracy: 85.16
[3,    83] loss: 0.41601, train_accuracy: 83.59
[3,    84] loss: 0.51086, train_accuracy: 82.81
[3,    85] loss: 0.41166, train_accuracy: 84.38
[3,    86] loss: 0.47988, train_accuracy: 86.72
[3,    87] loss: 0.54083, train_accuracy: 78.91
[3,    88] loss: 0.57408, train_accuracy: 82.81
[3,    89] loss: 0.39518, train_accuracy: 86.72
[3,    90] loss: 0.48227, train_accuracy: 80.47
[3,    91] loss: 0.37219, train_accuracy

[3,   242] loss: 0.48183, train_accuracy: 83.59
[3,   243] loss: 0.41562, train_accuracy: 85.94
[3,   244] loss: 0.45297, train_accuracy: 86.72
[3,   245] loss: 0.34392, train_accuracy: 87.50
[3,   246] loss: 0.58279, train_accuracy: 78.91
[3,   247] loss: 0.37382, train_accuracy: 89.06
[3,   248] loss: 0.26941, train_accuracy: 89.06
[3,   249] loss: 0.37677, train_accuracy: 89.06
[3,   250] loss: 0.57498, train_accuracy: 85.16
[3,   251] loss: 0.54554, train_accuracy: 82.03
[3,   252] loss: 0.55200, train_accuracy: 82.03
[3,   253] loss: 0.53162, train_accuracy: 82.81
[3,   254] loss: 0.24473, train_accuracy: 92.97
[3,   255] loss: 0.67899, train_accuracy: 79.69
[3,   256] loss: 0.59410, train_accuracy: 79.69
[3,   257] loss: 0.63307, train_accuracy: 76.56
[3,   258] loss: 0.45097, train_accuracy: 84.38
[3,   259] loss: 0.47619, train_accuracy: 82.81
[3,   260] loss: 0.61511, train_accuracy: 78.12
[3,   261] loss: 0.63656, train_accuracy: 75.78
[3,   262] loss: 0.48871, train_accuracy

,epoch,train_loss,train_accuracy,validation_loss,validation_accuracy,duration,criterion,optimizer,method,learning_rate,batchsize
0,1,2.413389,38.807545,1.304850,54.43,108.108993,CrossEntropyLoss(),Adam (\nParameter Group 0\n amsgrad: False\...,standard,NaN,80
1,2,1.134127,60.045556,0.990263,64.52,216.399744,CrossEntropyLoss(),Adam (\nParameter Group 0\n amsgrad: False\...,standard,NaN,80
2,3,0.865272,69.462916,0.899868,69.36,324.463973,CrossEntropyLoss(),Adam (\nParameter Group 0\n amsgrad: False\...,standard,NaN,80
3,1,0.866134,71.691176,0.764294,74.04,108.423787,CrossEntropyLoss(),Adam (\nParameter Group 0\n amsgrad: False\...,standard,NaN,80
4,2,0.622638,78.392743,0.738648,75.15,216.943161,CrossEntropyLoss(),Adam (\nParameter Group 0\n amsgrad: False\...,standard,NaN,80
5,3,0.505064,82.688219,0.676020,77.92,325.141385,CrossEntropyLoss(),Adam (\nParameter Group 0\n amsgrad: False\...,standard,NaN,80


In [6]:
model.train_stats['optimizer'][0]

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

# Full Pipeline - 152s

In [20]:
model = CifarResNet().to(device)
model.fit(train_loader, test_loader, 1, device)

identifying layers
[1,     1] loss: 2.78745, train_accuracy: 10.16
[1,     2] loss: 34.23767, train_accuracy: 9.38
[1,     3] loss: 23.95284, train_accuracy: 18.75
[1,     4] loss: 30.55729, train_accuracy: 13.28
[1,     5] loss: 36.58367, train_accuracy: 10.94
[1,     6] loss: 26.28638, train_accuracy: 6.25
[1,     7] loss: 13.82383, train_accuracy: 10.16
[1,     8] loss: 15.66329, train_accuracy: 14.06
[1,     9] loss: 12.64819, train_accuracy: 8.59
[1,    10] loss: 15.36391, train_accuracy: 7.81
[1,    11] loss: 7.47738, train_accuracy: 10.16
[1,    12] loss: 17.23159, train_accuracy: 9.38
[1,    13] loss: 14.77475, train_accuracy: 7.03
[1,    14] loss: 9.23299, train_accuracy: 7.03
[1,    15] loss: 9.45286, train_accuracy: 13.28
[1,    16] loss: 6.93368, train_accuracy: 13.28
[1,    17] loss: 6.41030, train_accuracy: 10.16
[1,    18] loss: 8.64443, train_accuracy: 10.94
[1,    19] loss: 3.52179, train_accuracy: 10.16
[1,    20] loss: 5.16632, train_accuracy: 17.19
[1,    21] loss: 

[1,   172] loss: 2.44553, train_accuracy: 22.66
[1,   173] loss: 2.74980, train_accuracy: 21.88
[1,   174] loss: 2.65715, train_accuracy: 21.88
[1,   175] loss: 2.53540, train_accuracy: 27.34
[1,   176] loss: 2.64625, train_accuracy: 21.88
[1,   177] loss: 2.26340, train_accuracy: 17.97
[1,   178] loss: 2.17109, train_accuracy: 17.19
[1,   179] loss: 2.14707, train_accuracy: 22.66
[1,   180] loss: 2.48100, train_accuracy: 20.31
[1,   181] loss: 2.62162, train_accuracy: 19.53
[1,   182] loss: 2.33696, train_accuracy: 25.78
[1,   183] loss: 2.10156, train_accuracy: 18.75
[1,   184] loss: 2.27609, train_accuracy: 21.09
[1,   185] loss: 2.40949, train_accuracy: 15.62
[1,   186] loss: 2.09517, train_accuracy: 23.44
[1,   187] loss: 2.39182, train_accuracy: 16.41
[1,   188] loss: 2.20529, train_accuracy: 18.75
[1,   189] loss: 2.35630, train_accuracy: 16.41
[1,   190] loss: 2.15628, train_accuracy: 28.12
[1,   191] loss: 1.94121, train_accuracy: 26.56
[1,   192] loss: 2.54936, train_accuracy

[1,   343] loss: 1.87129, train_accuracy: 36.72
[1,   344] loss: 1.88244, train_accuracy: 26.56
[1,   345] loss: 1.94002, train_accuracy: 31.25
[1,   346] loss: 1.81500, train_accuracy: 26.56
[1,   347] loss: 1.95406, train_accuracy: 21.09
[1,   348] loss: 1.88064, train_accuracy: 36.72
[1,   349] loss: 1.80671, train_accuracy: 35.16
[1,   350] loss: 2.00401, train_accuracy: 28.91
[1,   351] loss: 1.87571, train_accuracy: 34.38
[1,   352] loss: 1.92065, train_accuracy: 29.69
[1,   353] loss: 1.94781, train_accuracy: 28.12
[1,   354] loss: 1.99935, train_accuracy: 32.03
[1,   355] loss: 1.99637, train_accuracy: 28.12
[1,   356] loss: 1.88664, train_accuracy: 27.34
[1,   357] loss: 1.93833, train_accuracy: 27.34
[1,   358] loss: 1.91061, train_accuracy: 30.47
[1,   359] loss: 1.92077, train_accuracy: 26.56
[1,   360] loss: 1.80987, train_accuracy: 32.03
[1,   361] loss: 1.99050, train_accuracy: 27.34
[1,   362] loss: 1.82531, train_accuracy: 35.94
[1,   363] loss: 1.86083, train_accuracy

{'epochs_trained': 0,
 'avg_time_per_epoch': 154.96460223197937,
 'criterion': CrossEntropyLoss(),
 'optimizer': Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     eps: 1e-08
     lr: 0.001
     weight_decay: 0
 ),
 'hist': {'train_loss': [3.137745145641629],
  'train_accuracy': [22.95316496163683],
  'validation_loss': [1.734822559960281],
  'validation_accuracy': [37.5]},
 'val_accuracy': 37.5}

In [5]:
model.train_stats

{'train_loss_history': [2.8527273421397297],
 'train_accuracy_history': [33.750399616368284],
 'validation_loss_history': [1.5421439137639879],
 'validation_accuracy_history': [43.53],
 'epochs_trained': 1,
 'total_training_time': 108.13442301750183,
 'criterion': [CrossEntropyLoss()],
 'optimizer': [Adam (
  Parameter Group 0
      amsgrad: False
      betas: (0.9, 0.999)
      eps: 1e-08
      lr: 0.001
      weight_decay: 0
  )]}

# w/o color jitter - 127 s

In [13]:
model = CifarResNet().to(device)
model.fit(train_loader, test_loader, 1, device)

identifying layers
[1,     1] loss: 2.71906, train_accuracy: 10.94
[1,     2] loss: 31.83177, train_accuracy: 6.25
[1,     3] loss: 40.89853, train_accuracy: 5.47
[1,     4] loss: 38.26459, train_accuracy: 8.59
[1,     5] loss: 29.79355, train_accuracy: 13.28
[1,     6] loss: 29.99668, train_accuracy: 9.38
[1,     7] loss: 19.14003, train_accuracy: 13.28
[1,     8] loss: 24.36173, train_accuracy: 13.28
[1,     9] loss: 11.03250, train_accuracy: 8.59
[1,    10] loss: 13.41577, train_accuracy: 14.06
[1,    11] loss: 14.73741, train_accuracy: 8.59
[1,    12] loss: 11.69197, train_accuracy: 7.81
[1,    13] loss: 9.41380, train_accuracy: 10.94
[1,    14] loss: 9.56711, train_accuracy: 15.62
[1,    15] loss: 6.92797, train_accuracy: 17.19
[1,    16] loss: 4.90577, train_accuracy: 13.28
[1,    17] loss: 4.59638, train_accuracy: 17.97
[1,    18] loss: 6.84164, train_accuracy: 6.25
[1,    19] loss: 5.30594, train_accuracy: 18.75
[1,    20] loss: 5.90743, train_accuracy: 15.62
[1,    21] loss: 8

[1,   172] loss: 2.01363, train_accuracy: 28.91
[1,   173] loss: 2.10887, train_accuracy: 28.12
[1,   174] loss: 2.03718, train_accuracy: 28.91
[1,   175] loss: 2.05474, train_accuracy: 29.69
[1,   176] loss: 2.12229, train_accuracy: 26.56
[1,   177] loss: 2.06592, train_accuracy: 27.34
[1,   178] loss: 2.04574, train_accuracy: 28.12
[1,   179] loss: 1.99611, train_accuracy: 28.12
[1,   180] loss: 1.97262, train_accuracy: 31.25
[1,   181] loss: 2.09172, train_accuracy: 30.47
[1,   182] loss: 2.04191, train_accuracy: 30.47
[1,   183] loss: 1.96241, train_accuracy: 32.03
[1,   184] loss: 2.30822, train_accuracy: 21.88
[1,   185] loss: 2.15116, train_accuracy: 26.56
[1,   186] loss: 2.18135, train_accuracy: 26.56
[1,   187] loss: 1.98889, train_accuracy: 28.91
[1,   188] loss: 1.92764, train_accuracy: 31.25
[1,   189] loss: 1.83186, train_accuracy: 38.28
[1,   190] loss: 2.00745, train_accuracy: 21.09
[1,   191] loss: 1.91047, train_accuracy: 25.78
[1,   192] loss: 1.96792, train_accuracy

[1,   343] loss: 1.77375, train_accuracy: 39.06
[1,   344] loss: 1.69919, train_accuracy: 38.28
[1,   345] loss: 1.99508, train_accuracy: 29.69
[1,   346] loss: 1.82078, train_accuracy: 34.38
[1,   347] loss: 1.81249, train_accuracy: 32.81
[1,   348] loss: 1.94072, train_accuracy: 29.69
[1,   349] loss: 1.83569, train_accuracy: 39.06
[1,   350] loss: 2.00946, train_accuracy: 27.34
[1,   351] loss: 1.99693, train_accuracy: 27.34
[1,   352] loss: 1.86889, train_accuracy: 32.81
[1,   353] loss: 1.76471, train_accuracy: 34.38
[1,   354] loss: 1.68774, train_accuracy: 36.72
[1,   355] loss: 2.06402, train_accuracy: 25.78
[1,   356] loss: 1.96217, train_accuracy: 31.25
[1,   357] loss: 1.97939, train_accuracy: 25.78
[1,   358] loss: 1.87406, train_accuracy: 30.47
[1,   359] loss: 1.75787, train_accuracy: 41.41
[1,   360] loss: 1.62473, train_accuracy: 42.97
[1,   361] loss: 1.82472, train_accuracy: 33.59
[1,   362] loss: 1.87716, train_accuracy: 28.91
[1,   363] loss: 1.79778, train_accuracy

{'epochs_trained': 0,
 'avg_time_per_epoch': 127.58919072151184,
 'criterion': CrossEntropyLoss(),
 'optimizer': Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     eps: 1e-08
     lr: 0.001
     weight_decay: 0
 ),
 'hist': {'train_loss': [2.944924154549913],
  'train_accuracy': [25.872762148337596],
  'validation_loss': [1.6564999366108375],
  'validation_accuracy': [40.51]},
 'val_accuracy': 40.51}

# w/o random horizontal flip - 148 s

In [13]:
model = CifarResNet().to(device)
model.fit(train_loader, test_loader, 1, device)

identifying layers
[1,     1] loss: 2.73089, train_accuracy: 7.81
[1,     2] loss: 33.70625, train_accuracy: 6.25
[1,     3] loss: 37.72399, train_accuracy: 5.47
[1,     4] loss: 39.64744, train_accuracy: 13.28
[1,     5] loss: 23.97301, train_accuracy: 17.97
[1,     6] loss: 20.02452, train_accuracy: 17.19
[1,     7] loss: 13.37092, train_accuracy: 7.03
[1,     8] loss: 17.59218, train_accuracy: 14.06
[1,     9] loss: 16.38407, train_accuracy: 14.06
[1,    10] loss: 12.62568, train_accuracy: 14.06
[1,    11] loss: 9.25568, train_accuracy: 9.38
[1,    12] loss: 9.94829, train_accuracy: 14.06
[1,    13] loss: 9.08403, train_accuracy: 9.38
[1,    14] loss: 11.07196, train_accuracy: 12.50
[1,    15] loss: 11.03835, train_accuracy: 16.41
[1,    16] loss: 8.93029, train_accuracy: 12.50
[1,    17] loss: 7.73774, train_accuracy: 10.94
[1,    18] loss: 5.02678, train_accuracy: 17.97
[1,    19] loss: 5.41755, train_accuracy: 13.28
[1,    20] loss: 9.17142, train_accuracy: 10.16
[1,    21] loss:

[1,   172] loss: 1.85165, train_accuracy: 32.81
[1,   173] loss: 2.24087, train_accuracy: 28.12
[1,   174] loss: 2.27824, train_accuracy: 26.56
[1,   175] loss: 2.13371, train_accuracy: 28.12
[1,   176] loss: 1.97337, train_accuracy: 30.47
[1,   177] loss: 2.45703, train_accuracy: 20.31
[1,   178] loss: 2.11995, train_accuracy: 17.97
[1,   179] loss: 1.88462, train_accuracy: 30.47
[1,   180] loss: 2.17834, train_accuracy: 23.44
[1,   181] loss: 2.31661, train_accuracy: 20.31
[1,   182] loss: 1.97295, train_accuracy: 19.53
[1,   183] loss: 1.88052, train_accuracy: 27.34
[1,   184] loss: 1.97934, train_accuracy: 29.69
[1,   185] loss: 1.99207, train_accuracy: 30.47
[1,   186] loss: 2.10996, train_accuracy: 25.00
[1,   187] loss: 2.13081, train_accuracy: 28.91
[1,   188] loss: 1.80777, train_accuracy: 32.03
[1,   189] loss: 1.97322, train_accuracy: 29.69
[1,   190] loss: 2.19852, train_accuracy: 25.00
[1,   191] loss: 2.27146, train_accuracy: 30.47
[1,   192] loss: 2.19726, train_accuracy

[1,   343] loss: 1.89575, train_accuracy: 33.59
[1,   344] loss: 1.77417, train_accuracy: 41.41
[1,   345] loss: 1.89497, train_accuracy: 29.69
[1,   346] loss: 1.90319, train_accuracy: 28.91
[1,   347] loss: 1.90949, train_accuracy: 28.91
[1,   348] loss: 2.04251, train_accuracy: 36.72
[1,   349] loss: 2.00208, train_accuracy: 31.25
[1,   350] loss: 1.93692, train_accuracy: 28.12
[1,   351] loss: 1.95633, train_accuracy: 29.69
[1,   352] loss: 1.68713, train_accuracy: 34.38
[1,   353] loss: 1.94206, train_accuracy: 29.69
[1,   354] loss: 1.89232, train_accuracy: 35.16
[1,   355] loss: 2.07786, train_accuracy: 28.91
[1,   356] loss: 1.79529, train_accuracy: 36.72
[1,   357] loss: 1.72653, train_accuracy: 33.59
[1,   358] loss: 1.90266, train_accuracy: 35.94
[1,   359] loss: 2.05230, train_accuracy: 28.91
[1,   360] loss: 1.89385, train_accuracy: 29.69
[1,   361] loss: 1.93209, train_accuracy: 29.69
[1,   362] loss: 1.86382, train_accuracy: 32.03
[1,   363] loss: 1.90629, train_accuracy

{'epochs_trained': 0,
 'avg_time_per_epoch': 148.91602849960327,
 'criterion': CrossEntropyLoss(),
 'optimizer': Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     eps: 1e-08
     lr: 0.001
     weight_decay: 0
 ),
 'hist': {'train_loss': [3.107554952201941],
  'train_accuracy': [25.129475703324808],
  'validation_loss': [1.7050109467928922],
  'validation_accuracy': [39.74]},
 'val_accuracy': 39.74}

# w/o random rotation - 148 s

In [16]:
model = CifarResNet().to(device)
model.fit(train_loader, test_loader, 1, device)

identifying layers
[1,     1] loss: 2.59091, train_accuracy: 9.38
[1,     2] loss: 27.52983, train_accuracy: 7.81
[1,     3] loss: 35.05844, train_accuracy: 9.38
[1,     4] loss: 23.72497, train_accuracy: 18.75
[1,     5] loss: 20.20100, train_accuracy: 10.94
[1,     6] loss: 24.16389, train_accuracy: 9.38
[1,     7] loss: 28.31976, train_accuracy: 11.72
[1,     8] loss: 11.39920, train_accuracy: 8.59
[1,     9] loss: 8.72260, train_accuracy: 16.41
[1,    10] loss: 8.80157, train_accuracy: 14.84
[1,    11] loss: 13.42351, train_accuracy: 10.16
[1,    12] loss: 8.73238, train_accuracy: 11.72
[1,    13] loss: 7.69056, train_accuracy: 8.59
[1,    14] loss: 8.93891, train_accuracy: 12.50
[1,    15] loss: 7.53752, train_accuracy: 10.16
[1,    16] loss: 5.24707, train_accuracy: 7.81
[1,    17] loss: 4.76543, train_accuracy: 14.06
[1,    18] loss: 7.94311, train_accuracy: 12.50
[1,    19] loss: 5.75391, train_accuracy: 13.28
[1,    20] loss: 3.98639, train_accuracy: 19.53
[1,    21] loss: 4.8

[1,   172] loss: 2.35078, train_accuracy: 17.97
[1,   173] loss: 2.07586, train_accuracy: 25.78
[1,   174] loss: 1.88815, train_accuracy: 31.25
[1,   175] loss: 2.06590, train_accuracy: 31.25
[1,   176] loss: 2.06548, train_accuracy: 31.25
[1,   177] loss: 1.93422, train_accuracy: 30.47
[1,   178] loss: 2.03168, train_accuracy: 25.78
[1,   179] loss: 2.00081, train_accuracy: 25.00
[1,   180] loss: 1.90612, train_accuracy: 28.91
[1,   181] loss: 1.99527, train_accuracy: 28.91
[1,   182] loss: 2.04399, train_accuracy: 30.47
[1,   183] loss: 2.05088, train_accuracy: 23.44
[1,   184] loss: 2.12243, train_accuracy: 25.78
[1,   185] loss: 2.06697, train_accuracy: 26.56
[1,   186] loss: 1.92959, train_accuracy: 28.12
[1,   187] loss: 2.07570, train_accuracy: 25.00
[1,   188] loss: 2.33274, train_accuracy: 26.56
[1,   189] loss: 2.16263, train_accuracy: 25.78
[1,   190] loss: 1.90182, train_accuracy: 29.69
[1,   191] loss: 2.00371, train_accuracy: 21.09
[1,   192] loss: 2.03149, train_accuracy

[1,   343] loss: 2.12886, train_accuracy: 26.56
[1,   344] loss: 1.79190, train_accuracy: 28.91
[1,   345] loss: 1.92370, train_accuracy: 27.34
[1,   346] loss: 1.84822, train_accuracy: 39.06
[1,   347] loss: 1.98468, train_accuracy: 25.00
[1,   348] loss: 1.75195, train_accuracy: 38.28
[1,   349] loss: 1.91630, train_accuracy: 34.38
[1,   350] loss: 1.90386, train_accuracy: 25.00
[1,   351] loss: 1.81721, train_accuracy: 33.59
[1,   352] loss: 1.88080, train_accuracy: 34.38
[1,   353] loss: 2.06307, train_accuracy: 25.78
[1,   354] loss: 1.92496, train_accuracy: 25.78
[1,   355] loss: 1.68671, train_accuracy: 39.06
[1,   356] loss: 1.91359, train_accuracy: 28.91
[1,   357] loss: 1.86076, train_accuracy: 36.72
[1,   358] loss: 1.77974, train_accuracy: 40.62
[1,   359] loss: 1.88117, train_accuracy: 29.69
[1,   360] loss: 1.82426, train_accuracy: 34.38
[1,   361] loss: 1.98927, train_accuracy: 28.12
[1,   362] loss: 1.75256, train_accuracy: 40.62
[1,   363] loss: 1.87360, train_accuracy

{'epochs_trained': 0,
 'avg_time_per_epoch': 148.66814661026,
 'criterion': CrossEntropyLoss(),
 'optimizer': Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     eps: 1e-08
     lr: 0.001
     weight_decay: 0
 ),
 'hist': {'train_loss': [2.8230593899631744],
  'train_accuracy': [25.543478260869566],
  'validation_loss': [1.6168042074275921],
  'validation_accuracy': [42.76]},
 'val_accuracy': 42.76}

# w/o resizing - 142 s

In [18]:
model = CifarResNet().to(device)
model.fit(train_loader, test_loader, 1, device)

identifying layers
[1,     1] loss: 2.81770, train_accuracy: 7.81
[1,     2] loss: 35.36283, train_accuracy: 11.72
[1,     3] loss: 26.17254, train_accuracy: 21.09
[1,     4] loss: 20.82720, train_accuracy: 10.94
[1,     5] loss: 24.54180, train_accuracy: 8.59
[1,     6] loss: 15.62462, train_accuracy: 18.75
[1,     7] loss: 12.95870, train_accuracy: 7.03
[1,     8] loss: 13.69677, train_accuracy: 7.03
[1,     9] loss: 11.40793, train_accuracy: 7.81
[1,    10] loss: 9.86576, train_accuracy: 10.94
[1,    11] loss: 5.70351, train_accuracy: 13.28
[1,    12] loss: 7.70698, train_accuracy: 15.62
[1,    13] loss: 8.56297, train_accuracy: 14.06
[1,    14] loss: 10.54087, train_accuracy: 17.97
[1,    15] loss: 8.81648, train_accuracy: 12.50
[1,    16] loss: 4.83721, train_accuracy: 14.84
[1,    17] loss: 4.05362, train_accuracy: 11.72
[1,    18] loss: 4.59992, train_accuracy: 14.06
[1,    19] loss: 3.82780, train_accuracy: 10.94
[1,    20] loss: 5.90464, train_accuracy: 8.59
[1,    21] loss: 6

[1,   172] loss: 1.87000, train_accuracy: 32.81
[1,   173] loss: 1.69370, train_accuracy: 39.84
[1,   174] loss: 1.79453, train_accuracy: 30.47
[1,   175] loss: 1.94114, train_accuracy: 28.91
[1,   176] loss: 1.84333, train_accuracy: 29.69
[1,   177] loss: 1.81316, train_accuracy: 35.16
[1,   178] loss: 1.79244, train_accuracy: 33.59
[1,   179] loss: 1.49503, train_accuracy: 42.97
[1,   180] loss: 1.56331, train_accuracy: 40.62
[1,   181] loss: 1.77229, train_accuracy: 37.50
[1,   182] loss: 1.62251, train_accuracy: 39.06
[1,   183] loss: 2.12049, train_accuracy: 35.16
[1,   184] loss: 1.52983, train_accuracy: 45.31
[1,   185] loss: 1.85622, train_accuracy: 29.69
[1,   186] loss: 1.75090, train_accuracy: 34.38
[1,   187] loss: 1.84776, train_accuracy: 32.81
[1,   188] loss: 1.88282, train_accuracy: 33.59
[1,   189] loss: 1.53692, train_accuracy: 42.19
[1,   190] loss: 1.66074, train_accuracy: 42.97
[1,   191] loss: 1.71650, train_accuracy: 34.38
[1,   192] loss: 1.90489, train_accuracy

[1,   343] loss: 1.39060, train_accuracy: 50.00
[1,   344] loss: 1.48784, train_accuracy: 44.53
[1,   345] loss: 1.52151, train_accuracy: 47.66
[1,   346] loss: 1.54286, train_accuracy: 42.19
[1,   347] loss: 1.61188, train_accuracy: 36.72
[1,   348] loss: 2.10970, train_accuracy: 36.72
[1,   349] loss: 1.57273, train_accuracy: 47.66
[1,   350] loss: 2.04256, train_accuracy: 34.38
[1,   351] loss: 1.81987, train_accuracy: 29.69
[1,   352] loss: 1.75799, train_accuracy: 39.84
[1,   353] loss: 1.57756, train_accuracy: 38.28
[1,   354] loss: 1.77935, train_accuracy: 35.94
[1,   355] loss: 1.68459, train_accuracy: 44.53
[1,   356] loss: 1.68506, train_accuracy: 38.28
[1,   357] loss: 1.53355, train_accuracy: 46.88
[1,   358] loss: 1.80312, train_accuracy: 39.06
[1,   359] loss: 1.86189, train_accuracy: 37.50
[1,   360] loss: 1.58711, train_accuracy: 42.19
[1,   361] loss: 1.62852, train_accuracy: 40.62
[1,   362] loss: 1.92040, train_accuracy: 38.28
[1,   363] loss: 1.71332, train_accuracy

{'epochs_trained': 0,
 'avg_time_per_epoch': 142.11310338974,
 'criterion': CrossEntropyLoss(),
 'optimizer': Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     eps: 1e-08
     lr: 0.001
     weight_decay: 0
 ),
 'hist': {'train_loss': [2.4961614459371932],
  'train_accuracy': [33.35078324808184],
  'validation_loss': [1.5624872204623645],
  'validation_accuracy': [44.33]},
 'val_accuracy': 44.33}

# Color Jitter only - 139 s

In [3]:
model = CifarResNet().to(device)
model.fit(train_loader, test_loader, 1, device)

identifying layers
[1,     1] loss: 2.72934, train_accuracy: 10.16
[1,     2] loss: 32.79873, train_accuracy: 10.16
[1,     3] loss: 35.51935, train_accuracy: 16.41
[1,     4] loss: 31.12176, train_accuracy: 14.84
[1,     5] loss: 27.37873, train_accuracy: 10.16
[1,     6] loss: 18.06026, train_accuracy: 13.28
[1,     7] loss: 18.00992, train_accuracy: 4.69
[1,     8] loss: 23.75197, train_accuracy: 14.06
[1,     9] loss: 9.95253, train_accuracy: 7.03
[1,    10] loss: 11.40526, train_accuracy: 15.62
[1,    11] loss: 9.17352, train_accuracy: 11.72
[1,    12] loss: 8.48862, train_accuracy: 13.28
[1,    13] loss: 9.06450, train_accuracy: 19.53
[1,    14] loss: 13.53641, train_accuracy: 12.50
[1,    15] loss: 7.33773, train_accuracy: 14.06
[1,    16] loss: 9.71513, train_accuracy: 9.38
[1,    17] loss: 5.32476, train_accuracy: 10.94
[1,    18] loss: 8.41257, train_accuracy: 9.38
[1,    19] loss: 6.59242, train_accuracy: 10.16
[1,    20] loss: 7.11621, train_accuracy: 11.72
[1,    21] loss:

[1,   172] loss: 1.70902, train_accuracy: 45.31
[1,   173] loss: 2.03106, train_accuracy: 31.25
[1,   174] loss: 1.77800, train_accuracy: 34.38
[1,   175] loss: 1.68846, train_accuracy: 35.16
[1,   176] loss: 1.85306, train_accuracy: 32.81
[1,   177] loss: 1.82075, train_accuracy: 35.16
[1,   178] loss: 2.27571, train_accuracy: 29.69
[1,   179] loss: 1.78031, train_accuracy: 38.28
[1,   180] loss: 1.74826, train_accuracy: 36.72
[1,   181] loss: 1.54893, train_accuracy: 39.84
[1,   182] loss: 1.70221, train_accuracy: 42.19
[1,   183] loss: 1.78268, train_accuracy: 39.06
[1,   184] loss: 1.82487, train_accuracy: 42.19
[1,   185] loss: 1.86772, train_accuracy: 34.38
[1,   186] loss: 1.87836, train_accuracy: 32.81
[1,   187] loss: 1.63919, train_accuracy: 39.84
[1,   188] loss: 1.74640, train_accuracy: 36.72
[1,   189] loss: 1.75531, train_accuracy: 34.38
[1,   190] loss: 1.67848, train_accuracy: 43.75
[1,   191] loss: 1.87277, train_accuracy: 33.59
[1,   192] loss: 1.75054, train_accuracy

[1,   343] loss: 1.36201, train_accuracy: 52.34
[1,   344] loss: 1.59376, train_accuracy: 45.31
[1,   345] loss: 1.26007, train_accuracy: 56.25
[1,   346] loss: 1.39269, train_accuracy: 44.53
[1,   347] loss: 1.56342, train_accuracy: 42.19
[1,   348] loss: 1.36589, train_accuracy: 50.78
[1,   349] loss: 1.35425, train_accuracy: 52.34
[1,   350] loss: 1.47593, train_accuracy: 42.19
[1,   351] loss: 1.44143, train_accuracy: 47.66
[1,   352] loss: 1.46895, train_accuracy: 46.88
[1,   353] loss: 1.35691, train_accuracy: 51.56
[1,   354] loss: 1.41540, train_accuracy: 45.31
[1,   355] loss: 1.36098, train_accuracy: 57.03
[1,   356] loss: 1.56229, train_accuracy: 41.41
[1,   357] loss: 1.47370, train_accuracy: 46.88
[1,   358] loss: 1.41152, train_accuracy: 49.22
[1,   359] loss: 1.28601, train_accuracy: 50.78
[1,   360] loss: 1.52733, train_accuracy: 44.53
[1,   361] loss: 1.38292, train_accuracy: 46.88
[1,   362] loss: 1.40253, train_accuracy: 43.75
[1,   363] loss: 1.49416, train_accuracy

{'epochs_trained': 0,
 'avg_time_per_epoch': 139.21841287612915,
 'criterion': CrossEntropyLoss(),
 'optimizer': Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     eps: 1e-08
     lr: 0.001
     weight_decay: 0
 ),
 'hist': {'train_loss': [2.657406526758238],
  'train_accuracy': [34.67071611253197],
  'validation_loss': [1.396858785725847],
  'validation_accuracy': [49.11]},
 'val_accuracy': 49.11}

# Random Horizontal Flip only - 110s

In [7]:
model = CifarResNet().to(device)
model.fit(train_loader, test_loader, 1, device)

identifying layers
[1,     1] loss: 3.24445, train_accuracy: 10.16
[1,     2] loss: 33.16906, train_accuracy: 10.94
[1,     3] loss: 34.40232, train_accuracy: 10.94
[1,     4] loss: 20.02533, train_accuracy: 26.56
[1,     5] loss: 28.19654, train_accuracy: 17.19
[1,     6] loss: 19.47647, train_accuracy: 14.84
[1,     7] loss: 27.65862, train_accuracy: 8.59
[1,     8] loss: 22.47031, train_accuracy: 9.38
[1,     9] loss: 14.05315, train_accuracy: 17.97
[1,    10] loss: 17.44954, train_accuracy: 14.84
[1,    11] loss: 20.48391, train_accuracy: 8.59
[1,    12] loss: 10.64860, train_accuracy: 19.53
[1,    13] loss: 9.74438, train_accuracy: 16.41
[1,    14] loss: 17.34064, train_accuracy: 5.47
[1,    15] loss: 5.64611, train_accuracy: 10.94
[1,    16] loss: 7.96084, train_accuracy: 14.84
[1,    17] loss: 7.32646, train_accuracy: 23.44
[1,    18] loss: 3.86598, train_accuracy: 15.62
[1,    19] loss: 6.71867, train_accuracy: 17.97
[1,    20] loss: 4.52591, train_accuracy: 11.72
[1,    21] lo

[1,   173] loss: 1.91208, train_accuracy: 32.03
[1,   174] loss: 1.67884, train_accuracy: 38.28
[1,   175] loss: 1.83381, train_accuracy: 38.28
[1,   176] loss: 1.88408, train_accuracy: 33.59
[1,   177] loss: 1.75106, train_accuracy: 35.16
[1,   178] loss: 1.85911, train_accuracy: 28.91
[1,   179] loss: 1.70352, train_accuracy: 33.59
[1,   180] loss: 1.63405, train_accuracy: 45.31
[1,   181] loss: 1.86403, train_accuracy: 32.81
[1,   182] loss: 2.07888, train_accuracy: 33.59
[1,   183] loss: 1.93041, train_accuracy: 36.72
[1,   184] loss: 1.70821, train_accuracy: 31.25
[1,   185] loss: 1.95670, train_accuracy: 30.47
[1,   186] loss: 1.70071, train_accuracy: 37.50
[1,   187] loss: 2.05051, train_accuracy: 43.75
[1,   188] loss: 3.23305, train_accuracy: 25.78
[1,   189] loss: 1.70504, train_accuracy: 32.03
[1,   190] loss: 1.97098, train_accuracy: 34.38
[1,   191] loss: 1.82051, train_accuracy: 32.81
[1,   192] loss: 1.72065, train_accuracy: 38.28
[1,   193] loss: 1.87525, train_accuracy

[1,   344] loss: 1.55973, train_accuracy: 39.06
[1,   345] loss: 1.52950, train_accuracy: 45.31
[1,   346] loss: 1.73210, train_accuracy: 42.19
[1,   347] loss: 1.42356, train_accuracy: 50.78
[1,   348] loss: 1.46716, train_accuracy: 48.44
[1,   349] loss: 1.46108, train_accuracy: 49.22
[1,   350] loss: 1.48588, train_accuracy: 50.00
[1,   351] loss: 1.52567, train_accuracy: 46.88
[1,   352] loss: 1.47676, train_accuracy: 47.66
[1,   353] loss: 1.66771, train_accuracy: 45.31
[1,   354] loss: 1.30189, train_accuracy: 47.66
[1,   355] loss: 1.56412, train_accuracy: 38.28
[1,   356] loss: 1.52990, train_accuracy: 50.78
[1,   357] loss: 1.34111, train_accuracy: 49.22
[1,   358] loss: 1.43493, train_accuracy: 47.66
[1,   359] loss: 1.47432, train_accuracy: 41.41
[1,   360] loss: 1.55403, train_accuracy: 45.31
[1,   361] loss: 1.32644, train_accuracy: 51.56
[1,   362] loss: 1.36354, train_accuracy: 50.00
[1,   363] loss: 1.47405, train_accuracy: 38.28
[1,   364] loss: 1.52511, train_accuracy

{'epochs_trained': 0,
 'avg_time_per_epoch': 110.8130419254303,
 'criterion': CrossEntropyLoss(),
 'optimizer': Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     eps: 1e-08
     lr: 0.001
     weight_decay: 0
 ),
 'hist': {'train_loss': [2.672231035464255],
  'train_accuracy': [35.14585997442455],
  'validation_loss': [1.3972952728030048],
  'validation_accuracy': [49.13]},
 'val_accuracy': 49.13}

# Random Rotation only - 112 s

In [9]:
model = CifarResNet().to(device)
model.fit(train_loader, test_loader, 1, device)

identifying layers
[1,     1] loss: 3.01800, train_accuracy: 16.41
[1,     2] loss: 39.64801, train_accuracy: 11.72
[1,     3] loss: 56.67356, train_accuracy: 10.16
[1,     4] loss: 35.74637, train_accuracy: 9.38
[1,     5] loss: 24.94523, train_accuracy: 9.38
[1,     6] loss: 20.45514, train_accuracy: 13.28
[1,     7] loss: 18.60504, train_accuracy: 10.94
[1,     8] loss: 18.22313, train_accuracy: 7.81
[1,     9] loss: 8.29266, train_accuracy: 10.16
[1,    10] loss: 10.27006, train_accuracy: 12.50
[1,    11] loss: 17.30413, train_accuracy: 17.19
[1,    12] loss: 14.97085, train_accuracy: 11.72
[1,    13] loss: 5.05954, train_accuracy: 14.84
[1,    14] loss: 6.53913, train_accuracy: 21.88
[1,    15] loss: 3.46858, train_accuracy: 20.31
[1,    16] loss: 6.09013, train_accuracy: 14.84
[1,    17] loss: 5.78682, train_accuracy: 13.28
[1,    18] loss: 3.57312, train_accuracy: 17.19
[1,    19] loss: 5.53199, train_accuracy: 16.41
[1,    20] loss: 5.39329, train_accuracy: 5.47
[1,    21] loss

[1,   172] loss: 1.72704, train_accuracy: 38.28
[1,   173] loss: 2.03732, train_accuracy: 24.22
[1,   174] loss: 1.84323, train_accuracy: 28.12
[1,   175] loss: 1.79470, train_accuracy: 34.38
[1,   176] loss: 1.91841, train_accuracy: 27.34
[1,   177] loss: 1.67188, train_accuracy: 34.38
[1,   178] loss: 1.92275, train_accuracy: 32.81
[1,   179] loss: 1.55924, train_accuracy: 44.53
[1,   180] loss: 1.87209, train_accuracy: 35.16
[1,   181] loss: 1.64117, train_accuracy: 40.62
[1,   182] loss: 1.95828, train_accuracy: 28.91
[1,   183] loss: 1.70472, train_accuracy: 39.06
[1,   184] loss: 1.77927, train_accuracy: 39.06
[1,   185] loss: 1.88685, train_accuracy: 39.06
[1,   186] loss: 1.77597, train_accuracy: 31.25
[1,   187] loss: 1.79334, train_accuracy: 45.31
[1,   188] loss: 1.59424, train_accuracy: 39.06
[1,   189] loss: 1.54767, train_accuracy: 41.41
[1,   190] loss: 2.15524, train_accuracy: 35.16
[1,   191] loss: 1.65596, train_accuracy: 35.94
[1,   192] loss: 1.97502, train_accuracy

[1,   343] loss: 1.68676, train_accuracy: 33.59
[1,   344] loss: 1.77895, train_accuracy: 35.16
[1,   345] loss: 1.47791, train_accuracy: 48.44
[1,   346] loss: 1.53743, train_accuracy: 46.88
[1,   347] loss: 1.64767, train_accuracy: 39.84
[1,   348] loss: 1.56099, train_accuracy: 38.28
[1,   349] loss: 1.47818, train_accuracy: 46.88
[1,   350] loss: 1.51973, train_accuracy: 39.06
[1,   351] loss: 1.50908, train_accuracy: 46.88
[1,   352] loss: 1.59868, train_accuracy: 38.28
[1,   353] loss: 1.53806, train_accuracy: 46.09
[1,   354] loss: 1.82435, train_accuracy: 39.06
[1,   355] loss: 1.54157, train_accuracy: 41.41
[1,   356] loss: 1.58506, train_accuracy: 44.53
[1,   357] loss: 1.48275, train_accuracy: 44.53
[1,   358] loss: 1.50395, train_accuracy: 46.09
[1,   359] loss: 1.60244, train_accuracy: 42.97
[1,   360] loss: 1.73872, train_accuracy: 38.28
[1,   361] loss: 1.38962, train_accuracy: 54.69
[1,   362] loss: 1.46892, train_accuracy: 50.00
[1,   363] loss: 1.65700, train_accuracy

{'epochs_trained': 0,
 'avg_time_per_epoch': 112.65657949447632,
 'criterion': CrossEntropyLoss(),
 'optimizer': Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     eps: 1e-08
     lr: 0.001
     weight_decay: 0
 ),
 'hist': {'train_loss': [2.6833220775170092],
  'train_accuracy': [33.56817455242967],
  'validation_loss': [1.4091312885284424],
  'validation_accuracy': [49.41]},
 'val_accuracy': 49.41}

# Random Resize only - 119 s

In [11]:
model = CifarResNet().to(device)
model.fit(train_loader, test_loader, 1, device)

identifying layers
[1,     1] loss: 3.10206, train_accuracy: 12.50
[1,     2] loss: 25.93526, train_accuracy: 13.28
[1,     3] loss: 31.89803, train_accuracy: 8.59
[1,     4] loss: 18.16910, train_accuracy: 11.72
[1,     5] loss: 31.91439, train_accuracy: 11.72
[1,     6] loss: 26.85158, train_accuracy: 11.72
[1,     7] loss: 14.83936, train_accuracy: 10.94
[1,     8] loss: 11.17216, train_accuracy: 7.81
[1,     9] loss: 13.76136, train_accuracy: 14.84
[1,    10] loss: 16.65985, train_accuracy: 12.50
[1,    11] loss: 10.80370, train_accuracy: 11.72
[1,    12] loss: 9.83598, train_accuracy: 12.50
[1,    13] loss: 20.68222, train_accuracy: 12.50
[1,    14] loss: 10.29360, train_accuracy: 14.84
[1,    15] loss: 7.84886, train_accuracy: 10.94
[1,    16] loss: 8.45962, train_accuracy: 10.94
[1,    17] loss: 6.59555, train_accuracy: 12.50
[1,    18] loss: 5.21252, train_accuracy: 14.06
[1,    19] loss: 9.12383, train_accuracy: 8.59
[1,    20] loss: 4.50371, train_accuracy: 17.19
[1,    21] l

[1,   172] loss: 1.86145, train_accuracy: 32.03
[1,   173] loss: 1.98888, train_accuracy: 28.12
[1,   174] loss: 2.30943, train_accuracy: 27.34
[1,   175] loss: 2.12598, train_accuracy: 24.22
[1,   176] loss: 2.05125, train_accuracy: 28.12
[1,   177] loss: 1.78808, train_accuracy: 32.81
[1,   178] loss: 2.00980, train_accuracy: 30.47
[1,   179] loss: 1.95519, train_accuracy: 37.50
[1,   180] loss: 1.93661, train_accuracy: 28.12
[1,   181] loss: 1.78854, train_accuracy: 31.25
[1,   182] loss: 2.13988, train_accuracy: 29.69
[1,   183] loss: 1.91985, train_accuracy: 25.78
[1,   184] loss: 2.05615, train_accuracy: 35.94
[1,   185] loss: 1.84890, train_accuracy: 28.12
[1,   186] loss: 1.85268, train_accuracy: 33.59
[1,   187] loss: 1.82027, train_accuracy: 31.25
[1,   188] loss: 1.95259, train_accuracy: 28.12
[1,   189] loss: 1.94362, train_accuracy: 28.12
[1,   190] loss: 2.03577, train_accuracy: 28.12
[1,   191] loss: 1.82528, train_accuracy: 26.56
[1,   192] loss: 1.79208, train_accuracy

[1,   343] loss: 1.84307, train_accuracy: 26.56
[1,   344] loss: 1.81446, train_accuracy: 32.03
[1,   345] loss: 1.67665, train_accuracy: 36.72
[1,   346] loss: 1.80287, train_accuracy: 42.19
[1,   347] loss: 1.91096, train_accuracy: 35.16
[1,   348] loss: 1.82605, train_accuracy: 32.81
[1,   349] loss: 1.81246, train_accuracy: 37.50
[1,   350] loss: 1.79717, train_accuracy: 35.94
[1,   351] loss: 1.79764, train_accuracy: 32.03
[1,   352] loss: 1.92872, train_accuracy: 28.91
[1,   353] loss: 1.89364, train_accuracy: 32.03
[1,   354] loss: 1.79667, train_accuracy: 39.84
[1,   355] loss: 1.74719, train_accuracy: 32.81
[1,   356] loss: 1.88280, train_accuracy: 32.81
[1,   357] loss: 1.75367, train_accuracy: 32.03
[1,   358] loss: 1.82737, train_accuracy: 31.25
[1,   359] loss: 1.80144, train_accuracy: 34.38
[1,   360] loss: 1.94088, train_accuracy: 35.94
[1,   361] loss: 1.78906, train_accuracy: 36.72
[1,   362] loss: 1.85187, train_accuracy: 32.03
[1,   363] loss: 1.76019, train_accuracy

{'epochs_trained': 0,
 'avg_time_per_epoch': 119.81687951087952,
 'criterion': CrossEntropyLoss(),
 'optimizer': Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     eps: 1e-08
     lr: 0.001
     weight_decay: 0
 ),
 'hist': {'train_loss': [2.819187433518412],
  'train_accuracy': [27.552349744245525],
  'validation_loss': [1.6579050897042962],
  'validation_accuracy': [39.16]},
 'val_accuracy': 39.16}